In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as kr
import os, sys

In [3]:
import utils

In [4]:
#Loading the data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train.shape, test.shape

((42000, 785), (28000, 784))

In [5]:
#Preparing the data
x_train = train.drop(["label"], axis=1).astype("float32")
y_train = train["label"].astype("int32")
x_test = test.astype("float32")
x_train = (x_train.values.reshape(-1, 28, 28, 1))/255.0
x_test = (x_test.values.reshape(-1, 28, 28, 1))/255.0
y_train = kr.utils.to_categorical(y_train, 10)

In [6]:
x_train.shape, x_test.shape, y_train.shape

((42000, 28, 28, 1), (28000, 28, 28, 1), (42000, 10))

In [7]:
model = kr.models.Sequential([
    kr.layers.Dense(32, activation="relu", input_shape=(28, 28, 1)),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Dense(32, activation="relu"),
    kr.layers.Flatten(),
    kr.layers.Dense(10, activation="softmax")
])

model.summary()

optimizer = kr.optimizers.Adam(
    learning_rate= 0.001,
    name = "Adam"
)

model.compile(
    loss = "categorical_crossentropy",
    optimizer = optimizer,
    metrics = ["accuracy"]
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 28, 28, 32)        64        
_________________________________________________________________
dense_1 (Dense)              (None, 28, 28, 32)        1056      
_________________________________________________________________
dense_2 (Dense)              (None, 28, 28, 32)        1056      
_________________________________________________________________
dense_3 (Dense)              (None, 28, 28, 32)        1056      
_________________________________________________________________
dense_4 (Dense)              (None, 28, 28, 32)        1056      
_________________________________________________________________
dense_5 (Dense)              (None, 28, 28, 32)        1056      
_________________________________________________________________
dense_6 (Dense)              (None, 28, 28, 32)        1

In [8]:
print("Shapes:  ", y_train.shape, model.output_shape)
print("Model complexity:  ", model.count_params())

Shapes:   (42000, 10) (None, 10)
Model complexity:   259402


Actually I fit the model for 20 epochs and it spends 1600 sec!!!

In [11]:
model.fit(x = x_train, y = y_train, batch_size=50, epochs = 10)

Epoch 1/10
840/840 [==============================] - 79s 94ms/step - loss: 0.1544 - accuracy: 0.9536
Epoch 2/10
840/840 [==============================] - 81s 97ms/step - loss: 0.1430 - accuracy: 0.9559
Epoch 3/10
840/840 [==============================] - 83s 98ms/step - loss: 0.1354 - accuracy: 0.9589
Epoch 4/10
840/840 [==============================] - 88s 105ms/step - loss: 0.1259 - accuracy: 0.9613
Epoch 5/10
840/840 [==============================] - 88s 105ms/step - loss: 0.1205 - accuracy: 0.9618
Epoch 6/10
840/840 [==============================] - 89s 106ms/step - loss: 0.1105 - accuracy: 0.9653
Epoch 7/10
840/840 [==============================] - 91s 108ms/step - loss: 0.1038 - accuracy: 0.9665
Epoch 8/10
840/840 [==============================] - 83s 99ms/step - loss: 0.0979 - accuracy: 0.9682
Epoch 9/10
840/840 [==============================] - 88s 104ms/step - loss: 0.0860 - accuracy: 0.9726
Epoch 10/10
840/840 [==============================] - 90s 107ms/step - loss:

In [12]:
results = model.predict(x_test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")
image_id = pd.Series(range(1, 28001), name = "ImageId")
submission = pd.concat([image_id, results], axis = 1)
submission.to_csv("../output/practice1.csv", index=False)